In [1]:
#Import all the necessary libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
%matplotlib inline
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree

In [2]:
#Read given data into dataframes
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
target = train.Survived
#Let's combine train and test together
train.drop(['Survived'],inplace=True,axis=1)
whole = train.append(test)
train.corr()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Pclass,-0.035144,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,-0.549500,0.096067,0.159651,0.216225,1.000000


In [4]:
whole.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [5]:
whole['Title'] = whole['Name'].str.extract('([A-Za-z]+)\.')
whole.Title.value_counts()

Mr          757
Miss        260
Mrs         197
Master       61
Dr            8
Rev           8
Col           4
Major         2
Mlle          2
Ms            2
Don           1
Capt          1
Lady          1
Sir           1
Countess      1
Jonkheer      1
Dona          1
Mme           1
Name: Title, dtype: int64

In [6]:
Title_Dictionary = {
    "Don": 1,
    "Sir" : 1,
    "Jonkheer": 1,
    "Rev": 1,
    "Dona":2,
    "Lady" : 2,
    "Countess":2,
    "Dr": 3,
    "Capt": 3,
    "Col": 3,
    "Major": 3,
    "Mr" : 4,
    "Mme": 4,
    "Mrs" : 5,
    "Mlle": 5,
    "Ms": 6,
    "Miss" : 6,
    "Master" : 7
    
}

In [7]:
whole['Title'] = whole['Title'].map(Title_Dictionary)

In [8]:
Gender = {
    "male":1,
    "female":0
}

In [9]:
whole['Sex'] = whole['Sex'].map(Gender)

In [10]:
def getCabinClass(s):
    if s is np.nan:
        return 'U'
    else:
        return s[0]

In [11]:
whole['CabinClass'] = whole['Cabin'].apply(lambda c:getCabinClass(c))

In [12]:
whole['CabinClass'].value_counts()

U    1014
C      94
B      65
D      46
E      41
A      22
F      21
G       5
T       1
Name: CabinClass, dtype: int64

In [13]:
CabinMap = {
    'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7,'T':8,'U':0
}

In [14]:
whole['CabinClass'] = whole['CabinClass'].map(CabinMap)

In [15]:
whole.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,CabinClass
0,1,3,"Braund, Mr. Owen Harris",1,22,1,0,A/5 21171,7.2500,NaN,S,4,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38,1,0,PC 17599,71.2833,C85,C,5,3
2,3,3,"Heikkinen, Miss. Laina",0,26,0,0,STON/O2. 3101282,7.9250,NaN,S,6,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35,1,0,113803,53.1000,C123,S,5,3
4,5,3,"Allen, Mr. William Henry",1,35,0,0,373450,8.0500,NaN,S,4,0


In [16]:
EmbarkMap = {
    'S':1,'C':2,'Q':3
}

In [17]:
whole['Embarked'].fillna('S',inplace=True)

In [18]:
whole['Embarked'] = whole['Embarked'].map(EmbarkMap)

In [19]:
whole['Fare'].fillna(whole['Fare'].mean(),inplace=True)

In [20]:
whole['FamilySize'] = whole['SibSp'] + whole['Parch']

In [21]:
whole.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,CabinClass,FamilySize
0,1,3,"Braund, Mr. Owen Harris",1,22,1,0,A/5 21171,7.2500,NaN,1,4,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38,1,0,PC 17599,71.2833,C85,2,5,3,1
2,3,3,"Heikkinen, Miss. Laina",0,26,0,0,STON/O2. 3101282,7.9250,NaN,1,6,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35,1,0,113803,53.1000,C123,1,5,3,1
4,5,3,"Allen, Mr. William Henry",1,35,0,0,373450,8.0500,NaN,1,4,0,0


In [22]:
grouped_train = whole.iloc[:891].groupby(['Sex','Pclass','Title'])
grouped_median_train = grouped_train.median()
grouped_median_train = grouped_median_train.reset_index()[['Sex', 'Pclass', 'Title', 'Age']]
grouped_median_train

,Sex,Pclass,Title,Age
0,0,1,2,40.5
1,0,1,3,49.0
2,0,1,4,24.0
3,0,1,5,39.5
4,0,1,6,30.0
5,0,2,5,32.0
6,0,2,6,24.0
7,0,3,5,31.0
8,0,3,6,18.0
9,1,1,1,40.0


In [23]:
def fillAge(row):
    condition = (
        (grouped_median_train['Sex'] == row['Sex']) & 
        (grouped_median_train['Title'] == row['Title']) & 
        (grouped_median_train['Pclass'] == row['Pclass'])
    ) 
    return grouped_median_train[condition]['Age'].values[0]


def process_age():
    global whole
    # a function that fills the missing values of the Age variable
    whole['Age'] = whole.apply(lambda row: fillAge(row) if np.isnan(row['Age']) else row['Age'], axis=1)
    return whole

In [24]:
whole = process_age()

In [25]:
drop_features = ['PassengerId','Name','Ticket','Cabin','Fare']

In [26]:
whole.drop(drop_features,inplace=True,axis=1)

In [27]:
whole.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked,Title,CabinClass,FamilySize
0,3,1,22,1,0,1,4,0,1
1,1,0,38,1,0,2,5,3,1
2,3,0,26,0,0,1,6,0,0
3,1,0,35,1,0,1,5,3,1
4,3,1,35,0,0,1,4,0,0


In [28]:
x_tr,x_ts,y_tr,y_ts = train_test_split(whole[:891],target,test_size = 0.3,random_state = 3000)


In [29]:
sv = SVC()
sv.fit(x_tr,y_tr)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [30]:
accuracy_score(y_ts,sv.predict(x_ts))

0.8059701492537313

In [31]:
lr = LogisticRegression()
lr.fit(x_tr,y_tr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
accuracy_score(y_ts,lr.predict(x_ts))

0.8283582089552238

In [33]:
dt = tree.DecisionTreeClassifier()
dt.fit(x_tr,y_tr)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [34]:
accuracy_score(y_ts,dt.predict(x_ts))

0.8171641791044776

In [35]:
pred = dt.predict(whole[891:])

In [36]:
my_submission = pd.DataFrame({'PassengerId': test.index+892, 'Survived': pred})
# you could use any filename. We choose submission here
my_submission.to_csv('submission10.csv', index=False)